In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import random

#シード値の固定
def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

torch_fix_seed()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


データの読み込み

In [2]:
data_path = "/root/work/data/2021"
result_path = "/root/work/result/MIM/mask25"

In [3]:
transform = transforms.Resize((48,512))

x_data=[]

for i in range(1,302):
    img=np.load(file=data_path+"/deck_depth_map/data/hull"+str(i)+".npy")
    img=torch.tensor(img).reshape(1,48,521)
    img=transform(img)
    img=np.array(img[0])
    x_data.append(img)

x_data=np.array(x_data)
x_data = np.delete(x_data, range(117, 119), axis=0)
x_data=np.flip(x_data,axis=2)
x_data=np.array(x_data).reshape(299,1,48,512)
x_data=torch.tensor(x_data)

X_MAX = x_data.max()
x_data /= X_MAX

In [4]:
y =np.genfromtxt(data_path + "/hist_naked.dat", dtype=float, skip_header=1)
y = np.delete(y, range(117, 119), axis=0)
y_data=y[:,4:5]
#性能指数K

In [5]:
class HullDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, index):
        return self.data[index], self.targets[index]

In [6]:
import torch
import numpy as np
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.dataset import Subset
from sklearn.model_selection import KFold


batch_size=16

data=torch.tensor(x_data).float().clone().detach()
targets=torch.tensor(y_data).float().clone().detach()

dataset=HullDataset(data,targets)

kfold=KFold(n_splits=5,shuffle=True,random_state=0)

train_loaders=[]
val_loaders=[]

for train_index,val_index in kfold.split(dataset):
    train_dataset=Subset(dataset,train_index)
    val_dataset=Subset(dataset,val_index)

    train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
    val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)

    train_loaders.append(train_loader)
    val_loaders.append(val_loader)



/tmp/ipykernel_1894192/1316001768.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data=torch.tensor(x_data).float().clone().detach()


モデルの読み込み

In [7]:
import timm
import torch.nn as nn


In [8]:
from vessel_models.vessel_MAE_new import vessel_MAE_new

In [9]:
def fit(net, optimizer, num_epochs, mask_ratio, train_loader, test_loader, device, history,fold):

    # tqdmライブラリのインポート
    from tqdm.notebook import tqdm

    base_epochs = len(history)
  
    for epoch in range(base_epochs, num_epochs+base_epochs):

        # 1エポックあたりの累積損失(平均化前)
        train_fom_loss, train_mae_loss=0, 0
        val_fom_loss, val_mae_loss = 0, 0

        best_val_loss=1.0

        # 1エポックあたりのデータ累積件数
        n_train, n_test = 0, 0

        #訓練フェーズ
        net.train()
        # for inputs, labels in tqdm(train_loader):
        for inputs, labels in train_loader:
            # 1バッチあたりのデータ件数
            train_batch_size = len(labels)
            # 1エポックあたりのデータ累積件数
            n_train += train_batch_size
    
            # GPUヘ転送
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 勾配の初期化
            optimizer.zero_grad()

            # 予測計算
            fom_loss, mae_loss = net(inputs,labels,mask_ratio)

            # 損失計算
            # 割合とかは後で考える
            alpha=0.5
            # alpha=0
            # alpha = 1 / (1 + np.exp(-10 * ((epoch+1)/num_epochs - 0.5)))
            # alpha = 1 / (1 + np.exp(-20 * ((epoch+1)/num_epochs - 0.5)))
            loss = alpha * fom_loss + (1-alpha) * mae_loss

            # 勾配計算
            loss.backward()

            # パラメータ修正
            optimizer.step()

            # lossは平均計算が行われているので平均前の損失に戻して加算
            train_fom_loss += fom_loss.item() * train_batch_size
            train_mae_loss += mae_loss.item() * train_batch_size 
            

        #予測フェーズ
        net.eval()

        for inputs_test, labels_test in test_loader:
            # 1バッチあたりのデータ件数
            test_batch_size = len(labels_test)
            # 1エポックあたりのデータ累積件数
            n_test += test_batch_size

            # GPUヘ転送
            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            # 予測計算
            fom_loss_test, mae_loss_test = net(inputs_test,labels_test,mask_ratio)

            # 損失計算
            # fom_loss_test, mae_loss_test = criterion(outputs_test, labels_test)
            # loss_test = fom_loss_test + mae_loss_test 


            # lossは平均計算が行われているので平均前の損失に戻して加算
            val_fom_loss += fom_loss_test.item() * test_batch_size
            val_mae_loss += mae_loss_test.item() * test_batch_size

        # 損失計算
        rmse_train_fom_loss=(train_fom_loss / n_train)**0.5
        rmse_train_mae_loss=(train_mae_loss / n_train)**0.5
        rmse_val_fom_loss=(val_fom_loss / n_test)**0.5     
        rmse_val_mae_loss=(val_mae_loss / n_test)**0.5     
    

        # 結果表示
        if (epoch+1)%10==0:
            print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}],train_fom_loss: {rmse_train_fom_loss:.5f},val_fom_loss: {rmse_val_fom_loss:.5f},train_mae_loss: {rmse_train_mae_loss:.5f},val_mae_loss: {rmse_val_mae_loss:.5f}')
        # 記録
        item = np.array([epoch+1,
                        rmse_train_fom_loss,rmse_val_fom_loss,
                        rmse_train_mae_loss,rmse_val_mae_loss])

        history = np.vstack((history, item))

        save_path = result_path+'/model'+str(fold)+'.pth'

        if rmse_val_fom_loss<best_val_loss:
            best_val_loss=rmse_val_fom_loss
            # モデルの状態辞書を取得
            model_state = net.state_dict()
            # モデルの状態辞書を保存
            torch.save(model_state, save_path)
        

    return history

In [10]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

historys=[]
lr=0.0001
num_epochs=2000
mask_ratio=0.25


for fold in range(5):
    model=vessel_MAE_new()
    net=model.to(device)

    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    history = np.zeros((0,5))

    train_loader = train_loaders[fold]
    val_loader = val_loaders[fold]

    history = fit(net, optimizer, num_epochs, mask_ratio, train_loader, val_loader, device, history,fold)
    historys=np.append(historys,history)

Epoch [10/2000],train_fom_loss: 0.11651,val_fom_loss: 0.10268,train_mae_loss: 0.13232,val_mae_loss: 0.12290
Epoch [20/2000],train_fom_loss: 0.11008,val_fom_loss: 0.09527,train_mae_loss: 0.06497,val_mae_loss: 0.06382
Epoch [30/2000],train_fom_loss: 0.10886,val_fom_loss: 0.09977,train_mae_loss: 0.05212,val_mae_loss: 0.05095
Epoch [40/2000],train_fom_loss: 0.12776,val_fom_loss: 0.11427,train_mae_loss: 0.04639,val_mae_loss: 0.04255
Epoch [50/2000],train_fom_loss: 0.11509,val_fom_loss: 0.08542,train_mae_loss: 0.04126,val_mae_loss: 0.04222
Epoch [60/2000],train_fom_loss: 0.11526,val_fom_loss: 0.10411,train_mae_loss: 0.03729,val_mae_loss: 0.03732
Epoch [70/2000],train_fom_loss: 0.10294,val_fom_loss: 0.11623,train_mae_loss: 0.03647,val_mae_loss: 0.04181
Epoch [80/2000],train_fom_loss: 0.09556,val_fom_loss: 0.06636,train_mae_loss: 0.03588,val_mae_loss: 0.03509
Epoch [90/2000],train_fom_loss: 0.08385,val_fom_loss: 0.06405,train_mae_loss: 0.03446,val_mae_loss: 0.03463
Epoch [100/2000],train_fom_l

In [11]:
historys=historys.reshape(5,num_epochs,5)

In [12]:
np.save(result_path+'/log', historys)